# VAE

In [5]:
import sys
import os
sys.path.append( '..' )

from VAE.VAE_smac import *
from helpers.pc_stats import *

Available GPUs:  []


## Parameter definition

In [6]:
data_dir  = os.path.normpath(os.path.join(os.getcwd(), "../../runs/FIA/Com8_grown_together/oms"))
run_dir = os.path.normpath(os.path.join(os.getcwd(), "../../runs/VAE/hyperparameter_optimization"))
results_dir = os.path.normpath(os.path.join(os.getcwd(), "../../runs/VAE/results"))
test_configuration = False
overwrite = False
verbosity = 1
framework = "pytorch_jupyter"
outdir = Path(os.path.normpath(os.path.join(run_dir, f"smac_vae_{framework}")))

# Logging (time and steps)
last_timestamp = time.time()
step = 0
runtimes = {}


time_step(message="Setup loaded", verbosity=verbosity)

Setup loaded


In [7]:
X = read_data(data_dir, verbosity=verbosity)

configuration_space = ConfigurationSpace(name="LD", seed=42)
hyperparameters = [
    Constant(       "original_dim",             X.shape[1]),
    Float(          "input_dropout",            (0.0, 0.5), default=0.25),
    Integer(        "intermediate_layers",      (1, 5), default=2),
    Integer(        "intermediate_dimension",   (100, 200), log=True, default=200),
    Categorical(    "intermediate_activation",  ["relu", "selu", "tanh", "leakyrelu"], default="selu"),
    Integer(        "latent_dimension",         (10, 50), log=False, default=50),
    Categorical(    "solver",                   ["nadam"], default="nadam"),
    Float(          "learning_rate",            (1e-4, 1e-2), log=True, default=1e-3)
]
configuration_space.add_hyperparameters(hyperparameters)
forbidden_clauses = [
    ForbiddenGreaterThanRelation(configuration_space["latent_dimension"], configuration_space["intermediate_dimension"])
]
configuration_space.add_forbidden_clauses(forbidden_clauses)
if verbosity > 0: 
    print(f"Configuration space defined with estimated {configuration_space.estimate_size()} possible combinations.\n")

Data loaded
Configuration space defined with estimated inf possible combinations.



## Optimization definition

In [8]:
if "torch" in framework:
    device = search_device(verbosity=verbosity)
    fia_vae_hptune = FIA_VAE_hptune( X, test_size=0.2, configuration_space=configuration_space, model_builder=FIA_VAE,
                                        device=device, workers=0, batch_size=64, verbosity=verbosity )
elif "keras" in framework:
    fia_vae_hptune = FIA_VAE_hptune( X, test_size=0.2, configuration_space=configuration_space, model_builder=FIA_VAE,
                                        batch_size=64, verbosity=verbosity )
else:
    raise(ValueError(f"The framework {framework} is not implemented. The framework must contain one of ['torch', 'keras']."))

Using cpu device


In [9]:
scenario = Scenario( fia_vae_hptune.configuration_space, deterministic=True,
                     n_trials=20, min_budget=2, max_budget=100,
                     n_workers=1, output_directory=outdir,
                     walltime_limit=np.inf, cputime_limit=np.inf, trial_memory_limit=None )   # Max RAM in Bytes (not MB)
                    
initial_design = MultiFidelityFacade.get_initial_design(scenario, n_configs=10)
intensifier = Hyperband(scenario, incumbent_selection="highest_budget")
facade = MultiFidelityFacade( scenario, fia_vae_hptune.train, 
                              initial_design=initial_design, intensifier=intensifier,
                              overwrite=overwrite, logging_level=30-verbosity*10 )
time_step(message=f"SMAC defined. Overwriting: {overwrite}", verbosity=verbosity)

[INFO][abstract_initial_design.py:147] Using 5 initial design configurations and 0 additional configurations.
SMAC defined. Overwriting: False


### Search

In [10]:
if test_configuration:
    config = ConfigurationSpace(
                {'input_dropout': 0.25, 'intermediate_activation': 'leakyrelu', 'intermediate_dimension': 200,
                'intermediate_layers': 4, 'latent_dimension': 10, 'learning_rate': 0.001,
                'original_dim': 825000, 'solver': 'nadam'}
            )
    test_train(smac_model=fia_vae_hptune, config=config, verbosity=verbosity)
else:
    incumbent = run_optimization(facade=facade, smac_model=fia_vae_hptune, verbose_steps=10, verbosity=verbosity)

    results, runtimes = save_runhistory(incumbent=incumbent, fascade=facade, run_dir=run_dir, verbosity=verbosity)

'\nelse:\n    incumbent = run_optimization(facade=facade, smac_model=fia_vae_hptune, verbose_steps=10, verbosity=verbosity)\n\n    results, runtimes = save_runhistory(incumbent=incumbent, fascade=facade, run_dir=run_dir, verbosity=verbosity)\n'